In [1]:
import nltk
import numpy as np
import pandas as pd
import re
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import sklearn.model_selection
import sklearn.preprocessing as preproc
from sklearn.feature_extraction import text
import pickle
import warnings
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import euclidean_distances
from nltk.stem.porter import PorterStemmer
warnings.filterwarnings("ignore")
nltk.download('stopwords')
nltk.download('wordnet')

df = pd.read_csv('train_with_label.txt', sep='\t+')
df.columns = ['instance_id', 'sentence_1', 'sentence_2', 'gold_label']
df.shape
df.dtypes
df.info()
print(df.isnull().sum())

df2 = pd.read_csv('dev_with_label.txt', sep='\t+')
df2.columns = ['instance_id', 'sentence_1', 'sentence_2', 'gold_label']



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4076 entries, 0 to 4075
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instance_id  4076 non-null   object
 1   sentence_1   4076 non-null   object
 2   sentence_2   4076 non-null   object
 3   gold_label   4076 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 127.5+ KB
instance_id    0
sentence_1     0
sentence_2     0
gold_label     0
dtype: int64


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/clareschwarzenberg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/clareschwarzenberg/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}


In [3]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
        
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    # remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    # Tokenize each word
    text =  nltk.WordPunctTokenizer().tokenize(text)
        
    return text

In [4]:
df['Sentence_1_Token'] = list(map(clean_text, df.sentence_1))
df['Sentence_2_Token'] = list(map(clean_text, df.sentence_2))

df2['Sentence_1_Token'] = list(map(clean_text, df2.sentence_1))
df2['Sentence_2_Token'] = list(map(clean_text, df2.sentence_2))

In [5]:
lemm = nltk.stem.WordNetLemmatizer()
df['Sentence_1_Cleaned'] = list(map(lambda word:list(map(lemm.lemmatize, word)),df.Sentence_1_Token))
df['Sentence_2_Cleaned'] = list(map(lambda word:list(map(lemm.lemmatize, word)),df.Sentence_2_Token))

lemm = nltk.stem.WordNetLemmatizer()
df2['Sentence_1_Cleaned'] = list(map(lambda word:list(map(lemm.lemmatize, word)),df2.Sentence_1_Token))
df2['Sentence_2_Cleaned'] = list(map(lambda word:list(map(lemm.lemmatize, word)),df2.Sentence_2_Token))

In [6]:
stemmer = PorterStemmer()
df['Sentence_1_Cleaned'] = list(map(lambda word:list(map(stemmer.stem, word)),df.Sentence_1_Cleaned))
df['Sentence_2_Cleaned'] = list(map(lambda word:list(map(stemmer.stem, word)),df.Sentence_2_Cleaned))

df2['Sentence_1_Cleaned'] = list(map(lambda word:list(map(stemmer.stem, word)),df2.Sentence_1_Cleaned))
df2['Sentence_2_Cleaned'] = list(map(lambda word:list(map(stemmer.stem, word)),df2.Sentence_2_Cleaned))

In [7]:
df["Sentence_1_Length"] = df["sentence_1"].apply(lambda n: len(n.split()))
df["Sentence_2_Length"] = df["sentence_2"].apply(lambda n: len(n.split()))

df2["Sentence_1_Length"] = df2["sentence_1"].apply(lambda n: len(n.split()))
df2["Sentence_2_Length"] = df2["sentence_2"].apply(lambda n: len(n.split()))


In [8]:
df.head(3)

,instance_id,sentence_1,sentence_2,gold_label,Sentence_1_Token,Sentence_2_Token,Sentence_1_Cleaned,Sentence_2_Cleaned,Sentence_1_Length,Sentence_2_Length
0,train_id_1,The woman was exposed to the SARS virus while ...,The woman was exposed to the SARS virus while ...,1,"[woman, exposed, sars, virus, hospital, health...","[woman, exposed, sars, virus, hospital, health...","[woman, expos, sar, viru, hospit, health, care...","[woman, expos, sar, viru, hospit, health, care...",35,32
1,train_id_2,He said the problem needs to be corrected befo...,He said the prob lem needs to be corrected bef...,1,"[said, problem, needs, corrected, space, shutt...","[said, prob, lem, needs, corrected, space, shu...","[said, problem, need, correct, space, shuttl, ...","[said, prob, lem, need, correct, space, shuttl...",19,17
2,train_id_3,A representative for Phoenix-based U-Haul decl...,"Anthony Citrano , a representative for WhenU ,...",0,"[representative, phoenix, based, u, haul, decl...","[anthony, citrano, representative, whenu, decl...","[repres, phoenix, base, u, haul, declin, comme...","[anthoni, citrano, repres, whenu, declin, comm...",19,15


In [9]:
train, test = sklearn.model_selection.train_test_split(df, train_size = 0.7, random_state=42)


In [10]:
y_train = train['gold_label'].values
y_test = test['gold_label'].values

In [11]:
bow_transform = CountVectorizer(tokenizer=lambda doc: doc, ngram_range=[1,3], lowercase=False) 


In [12]:
X_train_bow = bow_transform.fit_transform(train['Sentence_1_Cleaned'], train['Sentence_2_Cleaned'])


In [13]:
X_test_1 = bow_transform.transform(test['Sentence_1_Cleaned'])
X_test_2 = bow_transform.transform(test['Sentence_2_Cleaned'])
X_test_bow = X_test_1 + X_test_2


In [14]:
tfidf_transform = text.TfidfTransformer(norm=None)
X_train_tfidf = tfidf_transform.fit_transform(X_train_bow)
X_test_tfidf = tfidf_transform.transform(X_test_bow)

In [15]:
coeff = range(1, 10)
param_grid = dict(C=coeff)

clf_lr = LogisticRegression(penalty='l2')

grid = GridSearchCV(clf_lr, param_grid, cv=5, scoring='accuracy')
grid.fit(X_train_bow, y_train)

print(grid.best_params_)

clf_lr = LogisticRegression(penalty='l2', C=grid.best_params_['C'])
clf_lr.fit(X_train_bow, y_train)

y_pred = clf_lr.predict(X_test_bow)

print(accuracy_score(y_test, y_pred))

{'C': 2}
0.7309893704006541


In [16]:
y_dev = df2['gold_label'].values

In [17]:
X_dev_bow_1 = bow_transform.transform(df2['Sentence_1_Cleaned'])
X_dev_bow_2 = bow_transform.transform(df2['Sentence_2_Cleaned'])
X_dev_bow = X_dev_bow_1 + X_dev_bow_2

In [18]:
tfidf_transform = text.TfidfTransformer(norm=None)
X_dev_tfidf = tfidf_transform.fit_transform(X_dev_bow)


In [19]:
y_dev_pred = clf_lr.predict(X_dev_bow)
print(accuracy_score(y_dev, y_dev_pred))


0.5836791147994468


In [20]:
from sklearn import svm

clfSVM = svm.SVC(kernel='linear')
clfSVM.fit(X_train_bow, y_train)
predictionSVM = clfSVM.predict(X_test_bow)
print(accuracy_score(y_test, predictionSVM))

0.7187244480784954


In [21]:
devSVM = clfSVM.predict(X_dev_bow)
print(accuracy_score(y_dev, devSVM))

0.598893499308437


In [22]:
df3 = pd.read_csv('test_without_label.txt', sep='\t+')
df3.columns = ['instance_id', 'sentence_1', 'sentence_2']

df3['Sentence_1_Token'] = list(map(clean_text, df3.sentence_1))
df3['Sentence_2_Token'] = list(map(clean_text, df3.sentence_2))

df3['Sentence_1_Cleaned'] = list(map(lambda word:list(map(lemm.lemmatize, word)),df3.Sentence_1_Token))
df3['Sentence_2_Cleaned'] = list(map(lambda word:list(map(lemm.lemmatize, word)),df3.Sentence_2_Token))

df2['Sentence_1_Cleaned'] = list(map(lambda word:list(map(stemmer.stem, word)),df2.Sentence_1_Cleaned))
df2['Sentence_2_Cleaned'] = list(map(lambda word:list(map(stemmer.stem, word)),df2.Sentence_2_Cleaned))


In [23]:
X_final_bow_1 = bow_transform.transform(df3['Sentence_1_Cleaned'])
X_final_bow_2 = bow_transform.transform(df3['Sentence_2_Cleaned'])
X_final_bow = X_final_bow_1 + X_final_bow_2

In [24]:
finalSVM = clfSVM.predict(X_final_bow)

In [25]:
file = open('ClareSchwarzenberg_test_result.txt', 'a')
for ind in df3.index:
    file.write(df3['instance_id'][ind])
    file.write("\t")
    file.write(str(finalSVM[ind]))
    file.write("\n")
file.close()
